In [ ]:
# default_exp hpar

# H Parameter
> A reader for the binary H parameter map

In [ ]:
# export
from pathlib import Path

import dask.array as da
import dask
import hvplot.xarray
import numpy as np
import xarray as xr
import holoviews as hv

In [ ]:
# export
basepath = Path("/luna4/maye/h_parameter_maps")
lons_path = basepath / "hpar_global128ppd_v1c.lon.bin"
lats_path = basepath / "hpar_global128ppd_v1c.lat.bin"
hpar_path = basepath / "hpar_global128ppd_v1c.bin"


def memmap_binary(fname):
    # one extra element for both lats (the 0-line) and lons (360==0)
    cols = 128 * 360 + 1
    rows = 128 * 140 + 1
    # this doesn't read, just maps the diskfile into a virtual array
    return np.memmap(fname, np.float32, mode="r", shape=(rows, cols), order="F")


def read_hpar_binary(fname=hpar_path):
    """Reading Paul Hayne's binary global 128 ppd H-parameter map.

    Parameters
    ----------
    fname: str, pathlib.Path
        Path to binary 128 ppd map file binary (ending in .bin)
    """
    if Path(fname).suffix != ".bin":
        print("not an expected filename ending in .bin .")
        return

    arr = memmap_binary(hpar_path)

    # create a dask array
    a = da.from_array(arr)

    # replacing the NAN values with actual np.nan
    nan_value = -32768
    a[a == nan_value] = np.nan

    # making up the lon/lat grid. Might need adaptation to match other global data
    lons = np.linspace(0, 360, arr.shape[1])
    lats = np.linspace(70, -70, arr.shape[0])

    # create xarray DataArray from the dask array (thankfully supported, but not documented)
    img = xr.DataArray(a, dims=["lat", "lon"], coords={"lat": lats, "lon": lons})
    img.name = "H"
    return img

Size in bytes:

In [ ]:
size = hpar_path.stat().st_size
size

Scaling down 80 lats to 70:

In [ ]:
n_lats = 20480 / 80 * 70
n_lats

In [ ]:
npix = 46080 * int(n_lats)
npix

multiply by 32 bits, divide by 8 to get bytes:

In [ ]:
expected_bytes = npix * 32 / 8
expected_bytes

Ratio:

In [ ]:
size / expected_bytes

In [ ]:
npix = 46081 * (int(n_lats) + 1)
npix

In [ ]:
expected_bytes = npix * 32 / 8
size / expected_bytes

In [ ]:
lats = memmap_binary(lats_path)
lats[0], lats[-1]

In [ ]:
lons = memmap_binary(lons_path)
lons[:, 0], lons[:, -1]

In [ ]:
with np.printoptions(precision=10):
    print(lons[0])

In [ ]:
with np.printoptions(precision=10):
    print(lons[:, 0])

In [ ]:
with np.printoptions(precision=20):
    print(lons[0])

In [ ]:
np.linspace(-70, 70, int(n_lats))

In [ ]:
# export
half_pixel_degree = 1 / 128 / 2
half_pixel_degree

In [ ]:
with_n_plus_1 = np.linspace(
    -70 - half_pixel_degree, 70 - half_pixel_degree, int(n_lats), endpoint=True
)

In [ ]:
from_data = lats[
    :,
    0,
][::-1]

In [ ]:
n = len(from_data)

In [ ]:
n

In [ ]:
%matplotlib widget

In [ ]:
from_data[:10]

In [ ]:
with_n_plus_1

In [ ]:
# export
class HReader:
    """Data reader class for H parameter map.

    It accesses preproduced data for 128 ppd for DEM, slope, and aspect,
    located on the luna4 disk.

    It uses virtual dask.Arrays so that virtually no memory is consumed until you
    resolve a chain of operations with the `.compute()` call.

    Attributes
    ----------
    H: xarray.DataArray
        H parameter map
    """

    def __init__(self, lat_limit=None, other_path=None):
        self.lat_limit = lat_limit
        path = hpar_path if other_path is None else other_path
        arr = memmap_binary(path)
        # dask array:
        a = da.from_array(arr[:, :-1])
        # replace NAN values with np.nan
        a[a == -32768] = np.nan

        # As per Paul's README:
        lats = np.linspace(70 - half_pixel_degree, -70 - half_pixel_degree, arr.shape[0], endpoint=True)
        lons = np.linspace(0+half_pixel_degree, 360+half_pixel_degree, arr.shape[1], endpoint=True)

        # create xarray DataArray from the dask array (thankfully supported, but not documented)
        img = xr.DataArray(a, dims=["lat", "lon"], coords={"lat": lats, "lon": lons[:-1]})
        img.name = "H"
        img.attrs["long_name"] = "H Parameter"
        img.attrs["units"] = "H units :-P"
        self.img = img

    def slice_lat(self, lat: float):  # Limiting latitude value
        """Return the map `data` constrained to lat <= `lat`"""
        s = slice(lat, -lat)
        return self.img.sel(lat=s, drop=True)

    def convert_to_lon180(self):
        "Switch image to -180/180 longitude system."
        p = self.img
        p.coords["lon"] = ((p.lon + 180) % 360) - 180
        with dask.config.set(**{"array.slicing.split_large_chunks": False}):
            self.img = p.sortby(p.lon)

    def convert_to_lon360(self):
        "Switch image to 360 longitude system."
        p = self.img
        p.coords["lon"] = p.coords["lon"] % 360
        with dask.config.set(**{"array.slicing.split_large_chunks": False}):
            self.img = p.sortby(p.lon)

    def assign_new_latlon(self, lat, lon):
        "make sure longitude layout matches!"
        p = self.img
        p.coords["lon"] = lon
        p.coords["lat"] = lat
        self.img = p.sortby(p.lon)

    def get_H_by_pixel(self, ilat, ilon):
        "Note: Decided to not apply offset!"
        val = self.img.isel(lat=ilat, lon=ilon)
        return float(val)

    def get_H_by_coord(self, lat, lon):
        "Note: Decided to not apply offset!"
        val = self.img.sel(lat=lat, lon=lon, method="nearest")
        return float(val)

    def plot(self, lat_min, lon_min, dlat=1, dlon=1, lat_max=None, lon_max=None):
        sliced = self.get_slice(lat_min, lon_min, dlat, dlon, lat_max, lon_max)
        return sliced.hvplot(cmap="viridis", aspect="equal", title="DEM")

    def get_slice(
        self, lat_min, lon_min, dlat=None, dlon=None, lat_max=None, lon_max=None
    ):
        """Slice a rectangular data tile out of the map.

        Parameters
        ----------
        lat_min: float
            Lower left corner latitude of slice.
        lon_min: float
            Lower left corner longitude of slice.
        dlat: float
            Delta to be added to `lat_min`
        dlon: float
            Delta lon to be added to `lon_min`
        lat_max: float
            Alternative for dlat, to set upper end of latitude interval.
        lon_max: float
            Alternative for dlon, to set right end of longitude interval.

        Returns
        -------
        xarray.DataArray
            Sliced for the provided coordinate values.
        """
        if lon_max is None:
            lon_max = lon_min + dlon
        if lat_max is None:
            lat_max = lat_min + dlat
        return self.img.sel(lat=slice(lat_max, lat_min), lon=slice(lon_min, lon_max))

In [ ]:
H = HReader()

In [ ]:
H.img

In [ ]:
H.img.lon

In [ ]:
H.img.isel(lon=slice(0,5), drop=True).hvplot()

In [ ]:
H.get_H_by_coord(20, 50)

In [ ]:
H.convert_to_lon360()

In [ ]:
H.img

In [ ]:
H.get_slice(20, 310, 1, 1)

In [ ]:
H.img.lon

In [ ]:
H.get_H_by_coord(20, 70)

In [ ]:
lons = memmap_binary(lons_path)

In [ ]:
lons.shape

In [ ]:
pd.Series(lons[:, 0]).value_counts()

In [ ]:
pd.Series(lons[:, -1]).value_counts()

In [ ]:
H = memmap_binary(hpar_path)

In [ ]:
H.shape

In [ ]:
H[:, 0][:10]

In [ ]:
H[:, -1][:10]

In [ ]:
H = read_hpar_binary()

In [ ]:
H.isel(lon=0).hvplot() * H.isel(lon=-1).hvplot()

In [ ]:
(H.isel(lon=0)[1:] - H.isel(lon=-1)[:-1]).compute().data

In [ ]:
left = H.isel(lon=-1)[:-1].compute().data

In [ ]:
right = H.isel(lon=0)[1:].compute().data

In [ ]:
pd.Series(left - right).value_counts()

In [ ]:
17326 / left.shape[0]

In [ ]:
%matplotlib widget

In [ ]:
fig, ax = plt.subplots()
ax.plot(H[:, 0], label="left")
ax.plot(H[:, -1], label="right")

In [ ]:
H[:, 0][:10]

In [ ]:
H[:, -1][:10]

In [ ]:
H.lon.diff("lon") / 2

In [ ]:
H = read_hpar_binary()

In [ ]:
import hvplot.xarray

In [ ]:
H.isel(lon=0).hvplot() * H.isel(lon=-1).hvplot()

In [ ]:
rate = 47

In [ ]:
rate * 8

In [ ]:
arr_x = []
arr_y = []
for i in range(5):
    arr_x.append(np.sort(np.random.uniform(size=20)))
    arr_y.append(np.sort(np.random.uniform(size=20)))

In [ ]:
plt.figure()
for x, y in zip(arr_x, arr_y):
    plt.plot(x, y)